# Self-Consistency Prompting  

Self-consistency prompting addresses the limitations of standard prompting by generating multiple independent reasoning paths and using consensus to determine the most reliable answer.  

**Self-Consistency Structure**:  

```
Input: [Problem] 
→ Reasoning Path 1 → Answer A
→ Reasoning Path 2 → Answer B  
→ Reasoning Path 3 → Answer C
→ Consensus Analysis → Final Answer
```

In [1]:
from mistralai import Mistral
from rich.console import Console
from rich.markdown import Markdown
from collections import Counter
import dotenv
import os

dotenv.load_dotenv()
MISTRAL_API_KEY = os.getenv("MISTRAL_API_KEY")

# Initialize Mistral client and rich console
client = Mistral(api_key=MISTRAL_API_KEY)
console = Console()

# Define the prompt
prompt = """
Solve this riddle: I speak without a mouth and hear without ears. I have no body, but I come alive with the wind. What am I? Provide a clear answer and explain your reasoning.
"""

# Generate multiple responses
responses = []
for _ in range(3):
    response = client.chat.complete(
        model="mistral-small-latest",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.7  # Add some randomness for variety
    )
    responses.append(response.choices[0].message.content)

# Simple aggregation: Pick the most common answer (or first if no exact matches)
answer_counts = Counter([r.split("Answer:")[1].split("\n")[0].strip() if "Answer:" in r else r for r in responses])
most_common_answer = max(answer_counts.items(), key=lambda x: x[1])[0]
final_response = next(r for r in responses if most_common_answer in r)

# Format and display
console.print(Markdown(f"# Self-Consistency Response (Riddle)\n\n{final_response}"))

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                                       Self-Consistency Response (Riddle)                                        ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛

The answer to the riddle is an echo.                                                                               

                                                   Explanation:                                                    

 1 "I speak without a mouth": An echo repeats sounds but does not have a physical mouth to produce them.           
 2 "Hear without ears": An echo responds to sounds it "hears" (detects) but does not have ears.                    
 3 "I have no body": An echo is not a physical object; it is a reflection of sound waves.                          
 4 "I come alive with the wind": While not strictly necessary for an echo, wind can carry sounds more effectively, 
   making echoes more noticeable in certain environments (like mountains or valleys).                              

Thus, the riddle describes the nature of an echo perfectly.

# Self-Consistency for Reasoning Tasks  

Self-consistency is great for logical or mathematical tasks where answers might vary due to model randomness. By sampling multiple responses, you can verify the correct solution.  


In [3]:
import re
from rich.panel import Panel 

reasoning_prompt = """
What is 15% of 80? Show your work and provide the final answer.
"""

# Generate multiple responses
responses = []
for _ in range(3):
    response = client.chat.complete(
        model="mistral-small-latest",
        messages=[{"role": "user", "content": reasoning_prompt}],
        temperature=0.5
    )
    responses.append(response.choices[0].message.content)

# Extract numerical answers for consistency check
numerical_answers = []
for r in responses:
    # Extract the final number (assuming it’s in the format "Final answer: X" or similar)
    match = re.search(r"\d+(\.\d+)?", r)
    if match:
        numerical_answers.append(match.group(0))

# Find the most common numerical answer
answer_counts = Counter(numerical_answers)
most_common_number = max(answer_counts.items(), key=lambda x: x[1])[0]
final_response = next(r for r in responses if most_common_number in r)

# Display
console.print(Panel(final_response, title="Self-Consistency Response (Math)", border_style="cyan"))

╭─────────────────────────────────────── Self-Consistency Response (Math) ────────────────────────────────────────╮
│ To find **15% of 80**, follow these steps:                                                                      │
│                                                                                                                 │
│ 1. **Convert the percentage to a decimal**:                                                                     │
│                                                                                                                 │
│    [                                                                                                            │
│    15\% = \frac{15}{100} = 0.15                                                                                 │
│    \]                                                                                                           │
│                                                                                                                 │
│ 2. **Multiply the decimal by the number**:                                                                      │
│                                                                                                                 │
│    [                                                                                                            │
│    0.15 \times 80 = 12                                                                                          │
│    \]                                                                                                           │
│                                                                                                                 │
│ 3. **Final Answer**:                                                                                            │
│                                                                                                                 │
│    [                                                                                                            │
│    \boxed{12}                                                                                                   │
│    \]                                                                                                           │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

# Self-Consistency for Creative Tasks  

For creative tasks like writing or brainstorming, self-consistency helps refine ideas by comparing multiple outputs and selecting the most coherent or combining elements for a polished result.  



In [4]:
creative_prompt = """
Create a catchy tagline for a lively coffee shop called "The Steam" Explain why the tagline works.
"""

# Generate multiple responses
responses = []
for _ in range(3):
    response = client.chat.complete(
        model="mistral-small-latest",
        messages=[{"role": "user", "content": creative_prompt}],
        temperature=0.9  # Higher temperature for creativity
    )
    responses.append(response.choices[0].message.content)

# Manual aggregation: Combine the best elements (simplified for demo)
# In practice, you might use similarity metrics or manual selection
taglines = [r.split("Tagline:")[1].split("\n")[0].strip() for r in responses if "Tagline:" in r]
final_tagline = taglines[0]  # Pick the first for simplicity (in practice, you’d compare quality)
final_response = next(r for r in responses if final_tagline in r)

console.print(Markdown(f"# Self-Consistency Response (Creative)\n\n{final_response}"))

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                                      Self-Consistency Response (Creative)                                       ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛

Tagline: "Brewing Joy, One Sip at a Time!"                                                                         

Why It Works:                                                                                                      

 1 Memorable & Upbeat – The alliteration ("Brewing Joy") makes it catchy and easy to remember, while "One Sip at a 
   Time" adds a cozy, inviting rhythm.                                                                             
 2 Emotional Connection – "Joy" taps into the warm, happy feelings people associate with great coffee, making the  
   shop feel welcoming and uplifting.                                                                              
 3 Simple & Clear – It quickly communicates what The Steam is about—quality coffee that brings happiness, without  
   being overly complicated.                                                                                       
 4 Encourages Engagement – The phrase invites customers to experience the joy for themselves, making them more     
   likely to visit.                                                                                                

This tagline perfectly captures the lively, energetic vibe of The Steam while making it feel like a must-visit spot
for coffee lovers! ☕✨

# Advanced Self-Consistency with Aggregation  

For complex tasks, self-consistency can involve aggregating responses (e.g., averaging numbers, synthesizing text) to create a robust final answer. This is like distilling the wisdom of multiple AI “brains.”  

In [7]:
aggregate_prompt = """
Estimate the cost of renovating a small kitchen (100 sq ft). Consider materials, labor, and permits. Provide a final cost range and explain your reasoning.
"""

# Generate multiple responses
responses = []
for i in range(3):
    response = client.chat.complete(
        model="mistral-small-latest",
        messages=[{"role": "user", "content": aggregate_prompt}],
        temperature=0.6
    )
    responses.append(response.choices[0].message.content)
    # Debug: Print raw response
    console.print(Panel(responses[-1], title=f"Raw Response {i+1}", border_style="yellow"))

# Extract cost ranges
cost_ranges = []
for r in responses:
    # Match ranges like $X,XXX-$Y,YYY or $X,XXX to $Y,YYY
    matches = re.findall(r"\$[\d,]+(?:\s*-\s*|\s*to\s*)\$[\d,]+", r)
    if matches:
        # Split on hyphen or "to" (normalize spaces)
        range_str = matches[0].replace(" to ", "-").strip()
        try:
            low, high = map(lambda x: int(x.replace("$", "").replace(",", "")), range_str.split("-"))
            cost_ranges.append((low, high))
        except ValueError as e:
            console.print(f"[red]Error parsing range in response: {matches[0]}[/red]")
            continue
    else:
        console.print("[red]No valid cost range found in response[/red]")

# Calculate average range or fallback
if cost_ranges:
    avg_low = sum(low for low, _ in cost_ranges) // len(cost_ranges)
    avg_high = sum(high for _, high in cost_ranges) // len(cost_ranges)
    final_range = f"${avg_low:,}-${avg_high:,}"
else:
    final_range = "No valid ranges found; check responses for single estimates."

# Pick a representative response (first valid one with a range)
final_response = next((r for r in responses if re.search(r"\$[\d,]+(?:\s*-\s*|\s*to\s*)\$[\d,]+", r)), responses[0])
final_response += f"\n\n**Aggregated Cost Range**: {final_range}"

# Display
console.print(Panel(final_response, title="Self-Consistency Response (Aggregated)", border_style="green"))

╭──────────────────────────────────────────────── Raw Response 1 ─────────────────────────────────────────────────╮
│ The cost of renovating a small kitchen (100 sq ft) can vary widely based on factors like materials, labor       │
│ rates, location, and the extent of the renovation. Below is a breakdown of estimated costs and a final range.   │
│                                                                                                                 │
│ ### **1. Materials (40-60% of total cost)**                                                                     │
│ - **Cabinets:** $2,000–$8,000 (stock vs. custom)                                                                │
│ - **Countertops:** $1,000–$4,000 (laminate to quartz/granite)                                                   │
│ - **Appliances:** $2,000–$6,000 (budget vs. high-end)                                                           │
│ - **Flooring:** $1,000–$3,000 (vinyl, tile, or hardwood)                                                        │
│ - **Backsplash:** $500–$2,000 (ceramic tile, glass, or mosaic)                                                  │
│ - **Lighting & Fixtures:** $500–$1,500                                                                          │
│ - **Plumbing & Electrical Upgrades:** $1,000–$3,000                                                             │
│ - **Paint & Trim:** $300–$1,000                                                                                 │
│                                                                                                                 │
│ **Total Materials Estimate:** **$8,000–$25,000**                                                                │
│                                                                                                                 │
│ ### **2. Labor (30-40% of total cost)**                                                                         │
│ - **Demolition & Prep:** $500–$2,000                                                                            │
│ - **Carpentry (cabinets, framing):** $2,000–$6,000                                                              │
│ - **Plumbing & Electrical Work:** $1,500–$4,000                                                                 │
│ - **Flooring Installation:** $500–$2,000                                                                        │
│ - **Countertop Installation:** $500–$1,500                                                                      │
│ - **Painting & Finishing:** $500–$1,500                                                                         │
│                                                                                                                 │
│ **Total Labor Estimate:** **$5,500–$17,000**                                                                    │
│                                                                                                                 │
│ ### **3. Permits & Contingency (5-10% of total cost)**                                                          │
│ - **Permits:** $200–$1,000 (varies by location)                                                                 │
│ - **Contingency (10% of total):** $1,000–$4,000                                                                 │
│                                                                                                                 │
│ **Total Permits & Contingency:** **$1,200–$5,000**                                                              │
│                                                                                                                 │
│ ### **Final Cost Range:**                                                                                       │
│ - **Budget Renovation (DIY + mid-range materials):** **$15,000–$25,000**                                        │
│ - **Mid-Range Renovation (professional labor + quality

╭──────────────────────────────────────────────── Raw Response 2 ─────────────────────────────────────────────────╮
│ Renovating a small kitchen (100 sq ft) involves several cost factors, including materials, labor, and permits.  │
│ Below is a breakdown of estimated costs and a final range.                                                      │
│                                                                                                                 │
│ ### **1. Materials (40-60% of total cost)**                                                                     │
│ - **Cabinetry**: $2,000–$8,000 (stock cabinets are cheaper; custom or semi-custom are more expensive).          │
│ - **Countertops**: $1,000–$4,000 (laminate is the cheapest; quartz or granite is mid-range; marble is premium). │
│ - **Appliances**: $2,000–$6,000 (budget models vs. high-end stainless steel).                                   │
│ - **Flooring**: $500–$2,500 (vinyl or laminate is affordable; hardwood or tile is pricier).                     │
│ - **Backsplash**: $300–$1,500 (peel-and-stick tiles are cheap; ceramic or glass is mid-range).                  │
│ - **Lighting & Fixtures**: $500–$2,000 (basic vs. designer fixtures).                                           │
│ - **Plumbing & Electrical**: $500–$2,000 (if relocating outlets or plumbing).                                   │
│ - **Paint & Finishing**: $200–$800.                                                                             │
│                                                                                                                 │
│ **Total Materials Cost**: **$7,000–$25,000**                                                                    │
│                                                                                                                 │
│ ### **2. Labor (30-50% of total cost)**                                                                         │
│ - **Demolition & Removal**: $500–$2,000                                                                         │
│ - **Carpentry (framing, drywall, cabinet installation)**: $2,000–$6,000                                         │
│ - **Plumbing & Electrical Work**: $1,500–$5,000                                                                 │
│ - **Flooring & Tile Installation**: $500–$2,500                                                                 │
│ - **Painting & Finishing**: $500–$1,500                                                                         │
│                                                                                                                 │
│ **Total Labor Cost**: **$5,000–$17,000**                                                                        │
│                                                                                                                 │
│ ### **3. Permits & Contingency (5-10% of total cost)**                                                          │
│ - **Permits**: $200–$1,000 (varies by location).                                                                │
│ - **Contingency (10%)**: $1,000–$4,000 (for unexpected issues).                                                 │
│                                                                                                                 │
│ **Total Permits & Contingency**: **$1,200–$5,000**                                                              │
│                                                                                                                 │
│ ### **Final Cost Range**                                                                                        │
│ - **Low-End (Budget-Friendly)**: **$13,000–$18,000**                                                            │
│   (Stock cabinets, laminate countertops, basic appliances, DIY labor where possible)                            │
│ - **Mid-Range (Standard Renovation)**: **$20,000–$35,0

╭──────────────────────────────────────────────── Raw Response 3 ─────────────────────────────────────────────────╮
│ The cost of renovating a small kitchen (100 sq ft) can vary significantly based on factors like location,       │
│ quality of materials, labor rates, and the extent of renovations. Below is a breakdown of estimated costs and a │
│ final range.                                                                                                    │
│                                                                                                                 │
│ ### **1. Materials (40-60% of total cost)**                                                                     │
│ - **Cabinetry**: $2,000–$10,000 (stock vs. custom)                                                              │
│ - **Countertops**: $1,000–$4,000 (laminate to quartz/granite)                                                   │
│ - **Appliances**: $2,000–$8,000 (budget vs. high-end)                                                           │
│ - **Flooring**: $500–$3,000 (vinyl to hardwood)                                                                 │
│ - **Backsplash**: $300–$1,500 (ceramic tile vs. premium materials)                                              │
│ - **Lighting & Fixtures**: $500–$2,000                                                                          │
│ - **Plumbing & Electrical**: $500–$2,000 (upgrades or relocations)                                              │
│ - **Paint & Finishes**: $300–$1,000                                                                             │
│                                                                                                                 │
│ **Total Materials Estimate**: **$7,100–$29,500**                                                                │
│                                                                                                                 │
│ ### **2. Labor (30-50% of total cost)**                                                                         │
│ - **Demolition & Prep**: $500–$2,000                                                                            │
│ - **Carpentry (cabinets, framing)**: $1,500–$5,000                                                              │
│ - **Plumbing & Electrical Work**: $1,000–$4,000                                                                 │
│ - **Flooring Installation**: $500–$2,000                                                                        │
│ - **Countertop & Backsplash Installation**: $500–$2,000                                                         │
│ - **Painting & Finishing**: $500–$1,500                                                                         │
│ - **General Contractor Fees (if applicable)**: $1,000–$5,000                                                    │
│                                                                                                                 │
│ **Total Labor Estimate**: **$5,000–$21,500**                                                                    │
│                                                                                                                 │
│ ### **3. Permits & Miscellaneous (5-10% of total cost)**                                                        │
│ - **Permits**: $200–$1,500 (varies by location)                                                                 │
│ - **Unexpected Costs (10% contingency)**: $1,000–$5,000                                                         │
│                                                                                                                 │
│ **Total Permits & Miscellaneous**: **$1,200–$6,500**                                                            │
│                                                                                                                 │
│ ### **Final Cost Range**                              

No valid cost range found in response

No valid cost range found in response

No valid cost range found in response

╭──────────────────────────────────── Self-Consistency Response (Aggregated) ─────────────────────────────────────╮
│ The cost of renovating a small kitchen (100 sq ft) can vary widely based on factors like materials, labor       │
│ rates, location, and the extent of the renovation. Below is a breakdown of estimated costs and a final range.   │
│                                                                                                                 │
│ ### **1. Materials (40-60% of total cost)**                                                                     │
│ - **Cabinets:** $2,000–$8,000 (stock vs. custom)                                                                │
│ - **Countertops:** $1,000–$4,000 (laminate to quartz/granite)                                                   │
│ - **Appliances:** $2,000–$6,000 (budget vs. high-end)                                                           │
│ - **Flooring:** $1,000–$3,000 (vinyl, tile, or hardwood)                                                        │
│ - **Backsplash:** $500–$2,000 (ceramic tile, glass, or mosaic)                                                  │
│ - **Lighting & Fixtures:** $500–$1,500                                                                          │
│ - **Plumbing & Electrical Upgrades:** $1,000–$3,000                                                             │
│ - **Paint & Trim:** $300–$1,000                                                                                 │
│                                                                                                                 │
│ **Total Materials Estimate:** **$8,000–$25,000**                                                                │
│                                                                                                                 │
│ ### **2. Labor (30-40% of total cost)**                                                                         │
│ - **Demolition & Prep:** $500–$2,000                                                                            │
│ - **Carpentry (cabinets, framing):** $2,000–$6,000                                                              │
│ - **Plumbing & Electrical Work:** $1,500–$4,000                                                                 │
│ - **Flooring Installation:** $500–$2,000                                                                        │
│ - **Countertop Installation:** $500–$1,500                                                                      │
│ - **Painting & Finishing:** $500–$1,500                                                                         │
│                                                                                                                 │
│ **Total Labor Estimate:** **$5,500–$17,000**                                                                    │
│                                                                                                                 │
│ ### **3. Permits & Contingency (5-10% of total cost)**                                                          │
│ - **Permits:** $200–$1,000 (varies by location)                                                                 │
│ - **Contingency (10% of total):** $1,000–$4,000                                                                 │
│                                                                                                                 │
│ **Total Permits & Contingency:** **$1,200–$5,000**                                                              │
│                                                                                                                 │
│ ### **Final Cost Range:**                                                                                       │
│ - **Budget Renovation (DIY + mid-range materials):** **$15,000–$25,000**                                        │
│ - **Mid-Range Renovation (professional labor + quality